In [11]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [12]:
driver = webdriver.Chrome(executable_path = "/usr/bin/chromedriver")

In [13]:
driver.get("https://www.ndtv.com/world-news")

In [14]:
news_items = driver.find_elements_by_class_name("news_Itm")
news_ads = driver.find_elements_by_class_name("adBg")

news_items = list(set(news_items) ^ set(news_ads))

In [15]:
len(news_items)

15

In [16]:
all_news = []

In [17]:
for news_item in news_items:
    news_item_img = news_item.find_element_by_class_name("news_Itm-img")
    news_link = news_item_img.find_element_by_tag_name("a")
    
    all_news.append(news_link.get_attribute("href"))

In [18]:
# Using only Selenium
# for i in range(len(news_items)):
#     try:
#         news_items = driver.find_elements_by_class_name("news_Itm")
#     #     wait = WebDriverWait(driver, 10)
#     #     items = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "news_Itm")))
#         heading = ""
#         description = ""
#         image_link = ""
#         content = ""
        
#         try:
#             news = news_items[i].find_element_by_class_name("news_Itm-img")
#             news_link = news.find_element_by_tag_name("a")
#             news_link.click()
#             heading = (driver.find_element_by_class_name("sp-ttl")).text

#             description = (driver.find_element_by_class_name("sp-descp")).text

#             image_wrap = driver.find_element_by_class_name("ins_instory_dv_cont")
#             image = image_wrap.find_element_by_tag_name("img")
#             image_link = image.get_attribute("src")

#             news_outerbody = driver.find_element_by_class_name("ins_storybody")
#             paragraphs = news_outerbody.find_elements_by_css_selector("> p")
#             content = []
#             for para in paragraphs:
#                 content.append(para.text)

#             content = "".join(content)

#             all_news.append((heading, description, image_link, content))
#             print("Successfully scraped news_item ", (i + 1))
#         except:
#             print("Panga hogya")
            
#     except:
#         print("Error in news item", (i + 1))
#     driver.back()

In [19]:
# Moving to next pages:
req = requests.get("https://www.ndtv.com/world-news")
soup = BeautifulSoup(req.text, "html.parser")

pages = soup.find(class_ = "listng_pagntn clear")

for anchor in pages.findAll("a")[1:10]:
    req = requests.get(anchor.attrs["href"])
    soup = BeautifulSoup(req.text, "html.parser")

    news_link_img = soup.find_all(class_ = "news_Itm-img")
    for news_link_img_itr in news_link_img:
        all_news.append(news_link_img_itr.a.attrs["href"])

In [20]:
len(all_news)

140

In [21]:
news_gathered = []

In [24]:
def gather_news(news_gathered, all_news):
    for link in all_news:
        req = requests.get(link)
        soup = BeautifulSoup(req.text, "html.parser")

        soup = BeautifulSoup(requests.get(link).text, "html.parser")

        heading = soup.find(class_ = "sp-ttl").text
        if(heading is None):
            heading = ""

        description = soup.find(class_ = "sp-descp").text
        if(description is None):
            description = ""

        img_wrap = soup.find(class_ = "ins_instory_dv_cont")
        if(img_wrap.img is None):
            image_link = ""
        else:
            image_link = img_wrap.img.attrs["src"]    

        news_outerbody = soup.find(class_ = "ins_storybody")
        content = ""
        if(news_outerbody is None):
            content = ""
        else:
            paragraphs = news_outerbody.find_all("p")    

            content = []
            for para in paragraphs:
                content.append(para.text)

            content = "".join(content)

        news_gathered.append(
            {
                "heading": heading,
                "description": description,
                "image_url": image_link,
                "content": content,
                "link": link
            }
        )

In [25]:
gather_news(news_gathered, all_news)

In [26]:
data = pd.DataFrame(news_gathered)

In [27]:
data

,heading,description,image_url,content,link
0,Vietnam Approves Cuba's Abdala Vaccine Amid I...,Abdala becomes the eighth COVID-19 vaccine ap...,https://c.ndtvimg.com/2021-09/fm7kuvo_abdala-v...,Only 6.3% of Vietnam's 98 million people have ...,https://www.ndtv.com/world-news/vietnam-approv...
1,"Afghanistan's Humanitarian Situation ""Pretty ...",More than 3.5 million have been displaced fro...,https://c.ndtvimg.com/2021-09/n7drh0os_afghani...,More than 3.5 million have been displaced from...,https://www.ndtv.com/world-news/afghanistans-h...
2,Watch: Ukulele Performance On Board SpaceX's ...,"Chris Sembroski, an US Air Force veteran, str...",https://c.ndtvimg.com/2021-09/kjjbco6o_spacex-...,SpaceX: Inspiration4 blasted off on Wednesday ...,https://www.ndtv.com/world-news/ukulele-perfor...
3,Australia Anti-Lockdown Protesters Pepper-Spr...,Australia anti-lockdown protest: Officers use...,https://c.ndtvimg.com/2021-09/r17n54u4_austral...,Australia Protest: Police said 6 officers were...,https://www.ndtv.com/world-news/australia-anti...
4,100 World Leaders In US Next Week For UN Even...,The UN General Assembly will look to build mo...,https://c.ndtvimg.com/2019-10/cc65avv_un-gener...,US President Joe Biden will address the UN Gen...,https://www.ndtv.com/world-news/world-leaders-...
...,...,...,...,...,...
135,Facebook Exempts VIPs From Some Of Its Rules:...,The article cites examples of posts from high...,https://c.ndtvimg.com/2020-08/0psfrbvg_faceboo...,"Facebook spokesman Andy Stone, in a series of ...",https://www.ndtv.com/world-news/facebook-exemp...
136,Joe Biden To Speak At UN General Assembly On ...,"Joe Biden, weakened after a chaotic US milita...",https://c.ndtvimg.com/2021-09/1l3kh3d8_joe-bid...,"Earlier Monday, the White House announced Joe ...",https://www.ndtv.com/world-news/us-president-j...
137,"Pak Involved In ""Harbouring Members Of Taliba...",Antony Blinken told the US House of Represent...,https://c.ndtvimg.com/2021-09/fo4ievhg_antony-...,Pakistan has had deep ties with Taliban and ha...,https://www.ndtv.com/world-news/afghanistan-cr...
138,England To Offer Covid Vaccines For 12-15 Age...,"The US, Israel and some European countries ha...",https://c.ndtvimg.com/2021-06/r6e5kmv8_vaccine...,COVID-19: Vulnerable children in the age brack...,https://www.ndtv.com/world-news/covid-19-engla...


In [68]:
data.to_csv("ndtv_world_18_sept")

In [69]:
driver.close()

In [1]:
import pandas as pd

In [2]:
data_india = pd.read_csv("ndtv_india_18_sept.csv")
data_world = pd.read_csv("ndtv_world_18_sept.csv")

In [3]:
data_india.head()

,Unnamed: 0,heading,description,image_url,content,link
0,0,"Rahul Gandhi's Solution To ""Punjabi Gordian K...","""Kudos to Rahul Gandhi for adopting Alexandri...",https://c.ndtvimg.com/2021-08/k58t2oog_rahul-g...,The Congress is working on solving a crisis in...,https://www.ndtv.com/india-news/rahul-gandhis-...
1,1,J&K Spending More Than Other States To Nurtur...,Lt Governor Manoj Sinha said the budget of Ja...,https://c.ndtvimg.com/2021-09/ikfarmms_manoj-s...,Manoj Sinha's comments came at the golden jubi...,https://www.ndtv.com/india-news/jammu-and-kash...
2,2,"PM Modi Says ""A Political Party Got Fever"" Se...","COVID-19 Vaccine India: ""People speak of feve...",https://c.ndtvimg.com/2021-09/bjm84p28_narendr...,"PM Modi said it was an ""emotional moment"" for ...",https://www.ndtv.com/india-news/pm-narendra-mo...
3,3,BJP To Hold 2-day Scheduled Caste Wings Meeti...,The programme will be inaugurated by BJP pres...,https://i.ndtvimg.com/i/2018-03/bjp-flag-gener...,As many as 150 people are expected to be prese...,https://www.ndtv.com/india-news/bjp-to-hold-tw...
4,4,Man Held In Mumbai With Links To Terror Modul...,The Delhi Police's Special Cell had on Tuesda...,https://i.ndtvimg.com/i/2016-04/arrest-generic...,The man was apprehended by Maharashtra ATS in ...,https://www.ndtv.com/india-news/man-held-in-mu...


In [4]:
data_india.drop(["image_url"], axis = 1, inplace = True)

In [6]:
data_world.drop(["image_url"], axis = 1, inplace = True)

In [8]:
data_india.to_csv("india.csv")
data_world.to_csv("world.csv")